# Importing Libraries
***

In [ ]:
import pandas as pd
import random
import os
import json
import random
import numpy as np
from scipy.stats import poisson
from neo4j import GraphDatabase
from pandas import DataFrame
import re
import csv
import ast

# Connecting to Neo4j
***

In [ ]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "123123123"))

# Deffining Useful Functions
***

In [ ]:
def run_query(query): 
     with driver.session() as session:
        result = session.run(query)
        #print(result.single()[0])
        
        
def delete_all_nodes():
    query = '''
    MATCH(n) DETACH DELETE(n)
    '''
    run_query(query)

def record_count():
    with driver.session() as session:
        result = session.run("MATCH (n) RETURN count(n) LIMIT 100")
        print(result.single())

def delete_nodes(var):
    query = f'''
    MATCH (c:{var})
    DELETE c

    '''
    run_query(query)

# Reading Raw Data
***

In [ ]:
# Define the path to the directory containing the JSON files
directory = r'C:\Users\odyky\Desktop\Property_Graph_Research_Publications\data'

# Create an empty list to store the DataFrames
dfs = []

# Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.json'):
        # Load the JSON file into a DataFrame
        file_path = os.path.join(directory, filename)
        print(file_path)
        df = pd.read_json(file_path)
        # Append the DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames in the list into a single DataFrame
df = pd.concat(dfs, ignore_index=True)

df.shape

C:\Users\odyky\Desktop\Property_Graph_Research_Publications\data\dblp_168.json
C:\Users\odyky\Desktop\Property_Graph_Research_Publications\data\dblp_169.json
C:\Users\odyky\Desktop\Property_Graph_Research_Publications\data\dblp_170.json
C:\Users\odyky\Desktop\Property_Graph_Research_Publications\data\dblp_171.json
C:\Users\odyky\Desktop\Property_Graph_Research_Publications\data\dblp_172.json
C:\Users\odyky\Desktop\Property_Graph_Research_Publications\data\dblp_245.json
C:\Users\odyky\Desktop\Property_Graph_Research_Publications\data\dblp_246.json


(70000, 23)

In [ ]:
df.to_csv('./data/merged.csv', encoding='utf-8', index=False)

In [ ]:
df = pd.read_csv('./data/merged.csv', encoding='utf-8')
df.head()

,id,title,doi,issue,keywords,lang,venue,year,n_citation,page_start,...,isbn,url,abstract,authors,doc_type,fos,indexed_abstract,v12_id,v12_authors,references
0,53e9b388b7602d9703e6d468,Improved assignment with ant colony optimizati...,10.1016/j.eswa.2011.01.134,8,"['multiple target tracking', 'np hard multidim...",en,{'raw': 'Expert Syst. Appl.'},2011,22,9172,...,NaN,['http://dx.doi.org/10.1016/j.eswa.2011.01.134...,Detecting and tracking ground targets is cruci...,"[{'id': '53f43421dabfaee1c0a87b9b', 'name': 'A...",Journal,[{'name': 'Ant colony optimization algorithms'...,"{'IndexLength': 187, 'InvertedIndex': {'been':...",1.994508e+09,"[{'name': 'Ali Onder Bozdogan', 'org': 'Ankara...","['558a43ede4b037c087561d22', '53e99a67b7602d97..."
1,53e9b388b7602d9703e6d3dc,Improved Bounds for Finger Search on a RAM,10.1007/978-3-540-39658-1_31,2,"['Data structures', 'Dictionary problem', 'Bal...",en,{'raw': 'european symposium on algorithms'},2013,47,249,...,NaN,['http://dx.doi.org/10.1007/s00453-012-9636-4'...,We present a new finger search tree with O(1) ...,"[{'id': '53f43b94dabfaee2a1d19007', 'name': 'A...",Journal,NaN,NaN,NaN,NaN,"['53e997ddb7602d9701fd2b44', '53e999d2b7602d97..."
2,53e9b388b7602d9703e6d137,Functorial Analysis of Algebraic Higher-Order ...,10.14279/tuj.eceasst.40.631.642,NaN,[],en,{'raw': 'ECEASST'},2010,4,NaN,...,NaN,['http://journal.ub.tu-berlin.de/eceasst/artic...,NaN,"[{'id': '53f4693ddabfaee4dc859903', 'name': 'U...",Journal,"[{'name': 'Mobile ad hoc network', 'w': 0.5444...","{'IndexLength': 173, 'InvertedIndex': {'modeli...",2.269296e+09,"[{'name': 'Ulrike Golas', 'org': 'Konrad-Zuse-...","['53e99b36b7602d97023d36c3', '53e99cf5b7602d97..."
3,53e9b388b7602d9703e6d518,Simulator sickness in virtual display gaming: ...,10.1145/1152215.1152263,NaN,"['simulator sickness symptom', 'ordinary game ...",en,{'raw': 'Mobile HCI'},2006,117,227,...,1-59593-390-5,"['http://dx.doi.org/10.1145/1152215.1152263', ...",In this paper we compare simulator sickness sy...,"[{'id': '53f430f0dabfaee4dc7467ae', 'name': 'J...",Conference,"[{'name': 'User experience design', 'w': 0.433...","{'IndexLength': 156, 'InvertedIndex': {'use': ...",2.102866e+09,"[{'name': 'Jukka Häkkinen', 'org': 'Nokia Rese...","['53e9ae84b7602d97038a2424', '53e9be2eb7602d97..."
4,53e9b388b7602d9703e6d417,Spectral Properties and Interpolation Error An...,10.1109/GLOCOM.2007.591,NaN,"['software radio', 'software defined radio', '...",en,{'raw': 'GLOBECOM'},2007,0,3123,...,978-1-4244-1043-9,"['http://dx.doi.org/10.1109/GLOCOM.2007.591', ...",The problem of variable sample rate conversion...,"[{'id': '53f430eadabfaedf43543535', 'name': 'A...",Conference,"[{'name': 'Nearest-neighbor interpolation', 'w...","{'IndexLength': 110, 'InvertedIndex': {'spectr...",2.130725e+09,"[{'name': 'A. Tkacenko', 'org': 'California In...","['5582c3e90cf2fcbbc5f1cbe8', '53e997b9b7602d97..."


In [ ]:
column_names = df.columns.tolist()
print(column_names)

['id', 'title', 'doi', 'issue', 'keywords', 'lang', 'venue', 'year', 'n_citation', 'page_start', 'page_end', 'volume', 'issn', 'isbn', 'url', 'abstract', 'authors', 'doc_type', 'fos', 'indexed_abstract', 'v12_id', 'v12_authors', 'references']


# Discarding Unnecessary Columns
***

In [ ]:
new_df = df.loc[:, ["id", "title", "doi", "keywords", "lang", "venue", "year", 'page_start', 'page_end', "volume", "abstract", "authors", "doc_type", "references"]]

In [ ]:
new_df.shape

(70000, 14)

# Removing Duplicates and Missing/Wrong Data
***

In [ ]:
new_df.drop_duplicates()
new_df.shape

(70000, 14)

In [ ]:
new_df.dropna(inplace=True)
new_df.shape

(37231, 14)

In [ ]:
# Creating a new column with pages of a paper
# Discarding columns page_start and page_end after
# convert string columns to numeric data type

new_df = new_df[pd.to_numeric(new_df['page_start'], errors='coerce').notnull()]
new_df = new_df[pd.to_numeric(new_df['page_end'], errors='coerce').notnull()]

# create a new column 'paper_pages' by subtracting 'page_start' from 'page_end'
new_df['paper_pages'] = new_df['page_end'].astype(float) - new_df['page_start'].astype(float)
new_df.drop(['page_start', 'page_end'], axis=1, inplace=True)
new_df.shape

(36000, 13)

In [ ]:
new_df = new_df[new_df['keywords'].astype(str) != '[]']
new_df.shape

(34524, 13)

In [ ]:
# Define a function to check if any author has no 'id' attribute
def has_no_id(authors):
    for author in eval(authors):
        if len(author['id']) < 3:
            return True
    return False

In [ ]:
# Filter out rows with authors that have no 'id' attribute
new_df = new_df[~new_df['authors'].apply(has_no_id)]
new_df.shape

(31309, 13)

In [ ]:
new_df.head()

,id,title,doi,keywords,lang,venue,year,volume,abstract,authors,doc_type,references,paper_pages
0,53e9b388b7602d9703e6d468,Improved assignment with ant colony optimizati...,10.1016/j.eswa.2011.01.134,"['multiple target tracking', 'np hard multidim...",en,{'raw': 'Expert Syst. Appl.'},2011,38,Detecting and tracking ground targets is cruci...,"[{'id': '53f43421dabfaee1c0a87b9b', 'name': 'A...",Journal,"['558a43ede4b037c087561d22', '53e99a67b7602d97...",6.0
1,53e9b388b7602d9703e6d3dc,Improved Bounds for Finger Search on a RAM,10.1007/978-3-540-39658-1_31,"['Data structures', 'Dictionary problem', 'Bal...",en,{'raw': 'european symposium on algorithms'},2013,66,We present a new finger search tree with O(1) ...,"[{'id': '53f43b94dabfaee2a1d19007', 'name': 'A...",Journal,"['53e997ddb7602d9701fd2b44', '53e999d2b7602d97...",37.0
5,53e9b388b7602d9703e6d185,Body Conformal Antennas for Superficial Hypert...,10.1109/TBME.2009.2029081,"['biological effects of fields', 'hyperthermia...",en,"{'raw': 'Biomedical Engineering, IEEE Transact...",2009,56,Hyperthermia is a powerful radiosensitizer for...,"[{'id': '53f43617dabfaee43ec2d8fb', 'name': 'D...",Journal,[],9.0
6,53e9b388b7602d9703e6d427,LLM: A Low Latency Messaging Infrastructure fo...,10.1007/3-540-36265-7_11,"['low overhead negative acknowledgment', 'tran...",en,{'raw': 'HiPC'},2002,2552,"In this paper, we develop a messaging infrastr...","[{'id': '53f79795dabfae9060acffeb', 'name': 'R...",Conference,"['53e9979eb7602d9701f6e8bd', '53e9a2acb7602d97...",11.0
9,53e9b388b7602d9703e6d557,The multi-terminal maximum-flow network-interd...,10.1016/j.ejor.2010.12.011,"['OR in military', 'Integer programming', 'Net...",en,{'raw': 'European Journal of Operational Resea...,2011,211,This paper defines and studies the multi-termi...,"[{'id': '53f352e8dabfae4b349503f9', 'name': 'I...",Journal,"['53e99804b7602d9702018cfc', '53e99b16b7602d97...",10.0


# Removing Journals with Missing/Wrong Volume Data
***

In [ ]:
new_df = new_df.loc[~((new_df['doc_type'] == 'Journal') & (new_df['volume'].isna()))]
new_df.shape

(31309, 13)

# Selecting 10000 Random Rows
***

In [ ]:
print(len(new_df[new_df['doc_type'] == "Conference"]))
print(len(new_df[new_df['doc_type'] == "Journal"]))

5406
25903


In [ ]:
sel = pd.concat([new_df[new_df['doc_type'] == "Conference"].sample(n = 5000), new_df[new_df['doc_type'] == "Journal"].sample(n = 5000)], ignore_index=True)

In [ ]:
sel.shape

(10000, 13)

# Generating Artificial References Data
***

In [ ]:
# Set the Poisson distribution parameter
lam = 8

# Define a function to generate a random list of IDs for each row
def generate_id_list(row):
    # Generate a random number of IDs using the Poisson distribution
    num_ids = poisson(lam).rvs()
    # Sample IDs from the original dataframe without replacement
    id_list = sel['id'].drop(row.name).sample(n=num_ids, replace=False).tolist()
    return id_list

# Apply the function to each row of the dataframe to generate the reference lists
sel['references'] = sel.apply(generate_id_list, axis=1)

In [ ]:
sel = sel[sel['references'].astype(str) != '[]']
sel.shape

(9996, 13)

# Removing Papers that Reference Themselves
***

In [ ]:
# Removing rows that contain the paper_id in the references column
for i, row in sel.iterrows():
    if row['id'] in row['references']: sel = sel.drop(i)

sel.reset_index()
sel.shape

(9996, 13)

In [ ]:
df_filtered = sel
df_filtered.shape

(9996, 13)

# Generating Artificial City Data
***

In [ ]:
# Generating a list of random cities
cities = ["Barcelona", "Madrid", "Athens", "Thessaloniki", "Berlin", "Munich", "London", "Paris", "Shanghai", "Moscow", "Valencia", "Lisbon", "Brighton", "Rome", "Napoli"]
cities_list = [cities[random.randrange(len(cities))] for i in range(df_filtered.shape[0])]
  
# Using 'City' as the column name
# and equating it to the list cities_list
df_filtered['City'] = cities_list

In [ ]:
df_filtered.shape

(9996, 14)

# Generating Artificial Reviewers Data
***

In [ ]:
authors_set = set()

for index, row in df_filtered.iterrows():
    authors = eval(row['authors'])
    for author in authors:
        id = author['id']
        authors_set.add(id)

print(authors_set)

{'5405f920dabfae91d30226be', '562ca3a745cedb3398c81d6b', '53f4614ddabfaec09f22bb3d', '53f43097dabfaee2a1ca578b', '53f42fcbdabfaedce54e3bd4', '53f43ed8dabfaedf435b8cd5', '562c9f9945cedb3398c7b98a', '53f46842dabfaee2a1db6896', '53f44c33dabfaee1c0b04644', '53f45970dabfaee0d9bffb71', '53f48320dabfaec09f2a482f', '53f458cadabfaedf43614c74', '562caa0445cedb3398c8bfcd', '53f43accdabfaee0d9b8ee82', '53f36baedabfae4b349ae454', '53f44b70dabfaedd74e02f58', '53f556c6dabfaea7cd1d811b', '562cb0c545cedb3398c96ead', '53f4382cdabfaeb22f481e6f', '53f44be4dabfaee02ad2f3a1', '53f385cedabfae4b34a11fe8', '54325312dabfaeb4ea4eb6c5', '562cae5945cedb3398c931fe', '53f4d0eddabfaef00ef8099f', '562ca33a45cedb3398c81406', '53f45159dabfaec09f1f37b1', '53f435b1dabfaeb2ac04d496', '53f44fcfdabfaeecd69d1e87', '53f42bfcdabfaedd74d21945', '5408712cdabfae450f41aa17', '53f815c4dabfae92b410a19c', '5430134bdabfaeca69bc3f57', '53f439e9dabfaeecd697f0b0', '54857117dabfae8a11fb2999', '53f42f58dabfaedd74d4fb52', '5408b075dabfae44f0

In [ ]:
def generate_reviewers(row):
    potential_reviewers = authors_set
    # Remove author ids from the potential reviewers
    authors = eval(row['authors'])
    for author in authors:
        potential_reviewers.discard(author['id'])
    if len(potential_reviewers) < 3:
        # If there are less than 3 potential reviewers, return an empty list
        return []
    else:
        # Randomly select 3 unique reviewer ids
        reviewers = random.sample(potential_reviewers, k=3)
        return reviewers

In [ ]:
# Apply the generate_reviewers function to the dataframe to create a new column
df_filtered['reviewers'] = df_filtered.apply(generate_reviewers, axis=1)

C:\Users\odyky\AppData\Local\Temp\ipykernel_23124\3748748427.py:12: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  reviewers = random.sample(potential_reviewers, k=3)


# Removing Double Quotes from Abstract Column
***

In [ ]:
df_filtered['abstract'] = df_filtered['abstract'].str.replace('"', '')

In [ ]:
df_filtered.shape

(9996, 15)

In [ ]:
df_filtered['doi'] = df_filtered['doi'].str.replace('"', '')

In [ ]:
df_filtered['title'] = df_filtered['title'].str.replace('"', '')

In [ ]:
df_filtered.head()

,id,title,doi,keywords,lang,venue,year,volume,abstract,authors,doc_type,references,paper_pages,City,reviewers
0,53e9b41bb7602d9703f14db9,Fingerprint recognition based on minutes group...,10.1007/978-3-642-29350-4_42,"['minutes group', 'new smart phone', 'fingerpr...",en,{'raw': 'ICAISC (2)'},2012,7268,In this paper author presents new solutions fo...,"[{'id': '53f326f7dabfae9a84476c90', 'name': 'M...",Conference,"[53e9b3bcb7602d9703ea4f58, 53e9b451b7602d9703f...",7.0,Thessaloniki,"[53f468dcdabfaeb22f54794f, 53f39bdadabfae4b34a..."
1,53e9b40eb7602d9703f04298,Parallel implementing of road situation modeli...,10.1007/11610496_82,"['urban traffic situation', 'inductive loop', ...",en,{'raw': 'APWeb Workshops'},2006,3842,Most traffic flow models are based on the traf...,"[{'id': '53f644a9dabfae57b759ee3e', 'name': 'Z...",Conference,"[53e9b3c8b7602d9703eb48d1, 53e9b457b7602d9703f...",4.0,Thessaloniki,"[53f784eedabfae92b40c203a, 562ca9cf45cedb3398c..."
2,53e9b3e2b7602d9703ecda43,Undirected graphs of entanglement 2,10.1007/978-3-540-77050-3_42,"['game theoretic idea', 'complexity measure', ...",en,{'raw': 'FSTTCS'},2007,4855,Entanglement is a complexity measure of direct...,"[{'id': '53f4367fdabfaee02accabd4', 'name': 'W...",Conference,"[53e9b3b7b7602d9703e9f7d9, 53e9b3fcb7602d9703e...",11.0,Moscow,"[543187e6dabfae8fa2cc731b, 53f3b413dabfae4b34b..."
3,53e9b409b7602d9703effbc0,Empirical Assessment of Software On-Line Diagn...,10.1007/3-540-40891-6_2,"['fault injection', 'normal testing', 'input s...",en,{'raw': 'SAFECOMP'},2000,1943,This paper is part of an on-going empirical re...,"[{'id': '5410fac4dabfae92b42b1802', 'name': 'J...",Conference,"[53e9b395b7602d9703e7793b, 53e9b3d4b7602d9703e...",12.0,Barcelona,"[53f43bd0dabfaedd74dd3598, 5609539945cedb3396e..."
4,53e9b3abb7602d9703e8e7d6,A gossip-based protocol to reach consensus via...,10.1007/978-3-540-68083-3_32,"['gossip-based protocol', 'fast convergence ch...",en,{'raw': 'GPC'},2008,5036,Gossip-based protocols for group communication...,"[{'id': '53f4637ddabfaeee22a48e4a', 'name': 'Q...",Conference,"[53e9b443b7602d9703f3f485, 53e9b43db7602d9703f...",11.0,Napoli,"[53f46725dabfaee2a1db1bd6, 53f438d6dabfaee0d9b..."


# Saving Preprocessed Data
***

In [ ]:
# Saving dataframe to csv
df_filtered.to_csv('./data/csvfile.csv', encoding='utf-8', index=False)

# Reading Preprocessed Data
***

In [ ]:
df = pd.read_csv("./data/csvfile.csv", encoding='utf-8')
df.head(5)

,id,title,doi,keywords,lang,venue,year,volume,abstract,authors,doc_type,references,paper_pages,City,reviewers
0,53e9b41bb7602d9703f14db9,Fingerprint recognition based on minutes group...,10.1007/978-3-642-29350-4_42,"['minutes group', 'new smart phone', 'fingerpr...",en,{'raw': 'ICAISC (2)'},2012,7268,In this paper author presents new solutions fo...,"[{'id': '53f326f7dabfae9a84476c90', 'name': 'M...",Conference,"['53e9b3bcb7602d9703ea4f58', '53e9b451b7602d97...",7.0,Thessaloniki,"['53f468dcdabfaeb22f54794f', '53f39bdadabfae4b..."
1,53e9b40eb7602d9703f04298,Parallel implementing of road situation modeli...,10.1007/11610496_82,"['urban traffic situation', 'inductive loop', ...",en,{'raw': 'APWeb Workshops'},2006,3842,Most traffic flow models are based on the traf...,"[{'id': '53f644a9dabfae57b759ee3e', 'name': 'Z...",Conference,"['53e9b3c8b7602d9703eb48d1', '53e9b457b7602d97...",4.0,Thessaloniki,"['53f784eedabfae92b40c203a', '562ca9cf45cedb33..."
2,53e9b3e2b7602d9703ecda43,Undirected graphs of entanglement 2,10.1007/978-3-540-77050-3_42,"['game theoretic idea', 'complexity measure', ...",en,{'raw': 'FSTTCS'},2007,4855,Entanglement is a complexity measure of direct...,"[{'id': '53f4367fdabfaee02accabd4', 'name': 'W...",Conference,"['53e9b3b7b7602d9703e9f7d9', '53e9b3fcb7602d97...",11.0,Moscow,"['543187e6dabfae8fa2cc731b', '53f3b413dabfae4b..."
3,53e9b409b7602d9703effbc0,Empirical Assessment of Software On-Line Diagn...,10.1007/3-540-40891-6_2,"['fault injection', 'normal testing', 'input s...",en,{'raw': 'SAFECOMP'},2000,1943,This paper is part of an on-going empirical re...,"[{'id': '5410fac4dabfae92b42b1802', 'name': 'J...",Conference,"['53e9b395b7602d9703e7793b', '53e9b3d4b7602d97...",12.0,Barcelona,"['53f43bd0dabfaedd74dd3598', '5609539945cedb33..."
4,53e9b3abb7602d9703e8e7d6,A gossip-based protocol to reach consensus via...,10.1007/978-3-540-68083-3_32,"['gossip-based protocol', 'fast convergence ch...",en,{'raw': 'GPC'},2008,5036,Gossip-based protocols for group communication...,"[{'id': '53f4637ddabfaeee22a48e4a', 'name': 'Q...",Conference,"['53e9b443b7602d9703f3f485', '53e9b43db7602d97...",11.0,Napoli,"['53f46725dabfaee2a1db1bd6', '53f438d6dabfaee0..."


# Deleting All Data from Neo4j
***

In [ ]:
#DELETE ALL NODES

delete_all_nodes()

# Creating and Loading Nodes
***

## Node Paper
### Creating and Loading Paper Nodes

In [ ]:
# Get unique papers

papers = df[['id','title', 'doi', 'abstract', 'paper_pages', 'year']]

papers.to_csv('./data/papers.csv', encoding='utf-8', index=False)

In [ ]:
query = '''
LOAD CSV WITH HEADERS FROM "file:///C:/Users/odyky/Desktop/Property_Graph_Research_Publications/data/papers.csv" AS x
CREATE (:Paper {title: x.title, id:x.id, DOI: x.doi, abstract:x.abstract, pages:x.pages, year: x.year})
'''

run_query(query)

## Node Author
### Creating and Loading Author Nodes

In [ ]:
#Get unique authors

authors_list = []
coauthors = []
corresponding_authors = {}
j = 0

for i in df['authors']:
    res = re.findall(r'{.*?}', i)
    for x in res:
        try:
            authors = eval(x)
        except:
            continue
        if df['id'][j] not in corresponding_authors:
            corresponding_authors[df['id'][j]] = authors['id']
        else:
            coauthors.append((df['id'][j],authors['id']))
        authors_list.append(authors)
    j = j+1

print(len(authors_list))
unique_data = {d['id']: d for d in authors_list}.values()

authors_df = pd.DataFrame(unique_data)
authors_df['id'] = authors_df['id'].str.replace('"', '')
authors_df['name'] = authors_df['name'].str.replace('"', '')
authors_df['org'] = authors_df['org'].str.replace('"', '')
authors_df.to_csv('./data/authors.csv', encoding='utf-8', index=False)

29642


In [ ]:
query = """
    LOAD CSV WITH HEADERS FROM "file:///C:/Users/odyky/Desktop/Property_Graph_Research_Publications/data/authors.csv" AS x
    MERGE (:Author {id: x.id, name: x.name})
    """
run_query(query=query)

## Node Keyword
### Creating and Loading Keyword Nodes

In [ ]:
df_keywords = df[['keywords']].drop_duplicates()

keywords_list = []
for i in df['keywords']:
    a = eval(i)
    keywords_list.extend(a)

#Get unique keywords
keywords_set = set(keywords_list)

# Convert set to list and MERGE a dataframe
keywords = pd.DataFrame(list(keywords_set), columns=['keyword'])
keywords.to_csv('./data/keywords.csv', encoding='utf-8', index=False)
keywords.shape

(65646, 1)

In [ ]:
query = """
    LOAD CSV WITH HEADERS FROM "file:///C:/Users/odyky/Desktop/Property_Graph_Research_Publications/data/keywords.csv" AS x
    MERGE (:Keyword {name: x.keyword})
    """
run_query(query=query)

## Node Conference
### Creating and Loading Conference Nodes

In [ ]:
conferences_ls = []

# Load Conference Nodes
with open('./data/csvfile.csv', encoding='utf-8', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['doc_type'] == 'Conference':
            conferences_ls.append(eval(row['venue'])['raw'])

conf_set = set(conferences_ls)
conf = pd.DataFrame(list(conferences_ls), columns=['conference'])

conf.to_csv('./data/conferences.csv', encoding='utf-8', index=False)
conf.shape

(4999, 1)

In [ ]:
query = """
    LOAD CSV WITH HEADERS FROM "file:///C:/Users/odyky/Desktop/Property_Graph_Research_Publications/data/conferences.csv" AS x
    MERGE (:Conference {name: x.conference})
    """
run_query(query=query)

## Node Journal
### Creating and Loading Journal Nodes

In [ ]:
journal_ls = []

# Load Journal Nodes
with open('./data/csvfile.csv', encoding='utf-8', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['doc_type'] == 'Journal':
            journal_ls.append(eval(row['venue'])['raw'])

jour_set = set(journal_ls)
jour = pd.DataFrame(list(journal_ls), columns=['journal'])

jour.to_csv('./data/journals.csv', encoding='utf-8', index=False)
jour.shape

(4997, 1)

In [ ]:
query = """
    LOAD CSV WITH HEADERS FROM "file:///C:/Users/odyky/Desktop/Property_Graph_Research_Publications/data/journals.csv" AS x
    MERGE (:Journal {name: x.journal})
    """
run_query(query=query)

In [ ]:
record_count()

<Record count(n)=106282>


# Creating and Loading Relationships
***

## Relationship Wrote (Author - Wrote -> Paper) 
### Creating and Loading Wrote Relationship
#### Containing Attribute: Corresponding_Author

In [ ]:
# MERGE CORRESPONDING AUTHORS AND COAUTHORS CSV PREVIOUS TO RELATIONS APPEND

df_pco = pd.DataFrame(coauthors, columns=['PaperID', 'CoauthorID'])

df_pco.to_csv('./data/papers_coauthors.csv', index=False)
print(df_pco.shape)

df_pcorr = pd.DataFrame(list(corresponding_authors.items()), columns=['PaperID', 'CorrespondingID'])

df_pcorr.to_csv('./data/papers_corresponding.csv', index=False)
df_pcorr.shape

(19653, 2)


(9989, 2)

In [ ]:
# CORRESPONDING AUTHORS
query = '''
LOAD CSV WITH HEADERS FROM "file:///C:/Users/odyky/Desktop/Property_Graph_Research_Publications/data/papers_corresponding.csv" AS x
MATCH (a:Author {id: x.CorrespondingID}), (b:Paper {id: x.PaperID})
MERGE (a)-[r:wrote {corresponding: True}]->(b)
'''
run_query(query)

In [ ]:
# COAUTHORS
query = '''
LOAD CSV WITH HEADERS FROM "file:///C:/Users/odyky/Desktop/Property_Graph_Research_Publications/data/papers_coauthors.csv" AS x
MATCH (a:Author {id: x.CoauthorID}), (b:Paper {id: x.PaperID})
MERGE (a)-[r:wrote {corresponding: False}]->(b)
'''

run_query(query)

## Relationship Has_Keyword (Paper - Has_Keyword -> Keyword) 
### Creating and Loading Has_Keyword Relationship

In [ ]:
# PAPER - KEYWORDS RELATIONSHIP

paper_keywords = {}
for i, row in df.iterrows():
    paper = row['id']
    paper_keywords[f'{paper}'] = ast.literal_eval(row['keywords'])

In [ ]:
with open('./data/paper_keyword.csv', 'w', encoding='utf-8', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['paper_id', 'keyword'])
    for key, value in paper_keywords.items():
        for v in value:
            writer.writerow([key, v])

In [ ]:
#LOAD PAPER-KEYWORD RELATIONSHIP
query = """
    LOAD CSV WITH HEADERS FROM "file:///C:/Users/odyky/Desktop/Property_Graph_Research_Publications/data/paper_keyword.csv" AS x
    WITH x, x.paper_id AS paper_id
    MATCH (a:Paper {id: paper_id}), (b:Keyword {name: x.keyword})
    MERGE (a)-[r:has_keyword]->(b)
"""
run_query(query)

## Relationship Published_in_journal (Paper - Published_in_journal -> Journal) 
### Creating and Loading Published_in_journal Relationship
#### Containing Attribute: Volume, Year

In [ ]:
#LOAD PAPER-JOURNAL RELATIONSHIP
paper_journal = {}
journal_df = df[df['doc_type'] == 'Journal']
for i, row in journal_df.iterrows():
    paper = row['id']
    lst = [ast.literal_eval(row['venue'])['raw'], row['volume'], row['year']]
    paper_journal[f'{paper}'] = lst

In [ ]:
with open('./data/paper_journal.csv', 'w', encoding='utf-8', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['paper_id', 'journal', 'volume', 'year'])
    for key, value in paper_journal.items():
        journal = value[0]
        volume = value[1]
        year = value[2]
        writer.writerow([key, journal, volume, year])

In [ ]:
query = """
    LOAD CSV WITH HEADERS FROM "file:///C:/Users/odyky/Desktop/Property_Graph_Research_Publications/data/paper_journal.csv" AS x
    WITH x, x.paper_id AS paper_id
    MATCH (a:Paper {id: paper_id}), (b:Journal {name: x.journal})
    MERGE (a)-[r:published_in_journal {volume: x.volume, year: x.year}]->(b)
"""
run_query(query=query)

## Relationship Published_in_Conference (Paper - Published_in_conference -> Conference) 
### Creating and Loading Published_in_Conference Relationship
#### Containing Attribute: City, Year

In [ ]:
#LOAD PAPER-CONFERENCE RELATIONSHIP
paper_conference = {}
conference_df = df[df['doc_type'] == 'Conference']
for index, row in conference_df.iterrows():
    paper = row['id']
    lst = [ast.literal_eval(row['venue'])['raw'], row['City'], row['year']]
    paper_conference[f'{paper}'] = lst

In [ ]:
with open('./data/paper_conference.csv', 'w', encoding='utf-8', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['paper_id', 'conference', 'city', 'year'])
    for key, value in paper_conference.items():
        conference = value[0]
        city = value[1]
        year = value[2]
        writer.writerow([key, conference, city, year])

In [ ]:
query = """
    LOAD CSV WITH HEADERS FROM "file:///C:/Users/odyky/Desktop/Property_Graph_Research_Publications/data/paper_conference.csv" AS x
    WITH x, x.paper_id AS paper_id
    MATCH (a:Paper {id: paper_id}), (b:Conference {name: x.conference})
    MERGE (a)-[r:published_in_conference {city: x.city, year: x.year}]->(b)
"""
run_query(query=query)

## Relationship Reviews (Author - Reviews -> Paper) 
### Creating and Loading Reviews Relationship

In [ ]:
#MERGE RELATION REVIEWS

papers_reviewers = []

j = 0
for i in df['reviewers']:
    a = eval(i)
    for el in a:
        papers_reviewers.append((df['id'][j],el))
    j = j + 1

#MERGE PAPERS ID AND PAPERS CITATIONS CSV PREVIOUS TO RELATIONS APPEND

df_rev = pd.DataFrame(papers_reviewers, columns=['PaperID', 'ReviewerID'])

df_rev.to_csv('./data/papers_reviewers.csv', index=False)

In [ ]:
#REVIEWS
query = '''
LOAD CSV WITH HEADERS FROM "file:///C:/Users/odyky/Desktop/Property_Graph_Research_Publications/data/papers_reviewers.csv" AS x
MATCH (a:Author {id: x.ReviewerID}), (b:Paper {id: x.PaperID})
MERGE (a)-[:reviews]->(b)
'''

run_query(query)

## Relationship Cites (Paper - Cites -> Paper) 
### Creating and Loading Cites Relationship

In [ ]:
#MERGE RELATION CITES

papers_citations = []

j = 0
for i in df['references']:
    a = eval(i)
    for el in a:
        papers_citations.append((df['id'][j],el))
    j = j + 1

#MERGE PAPERS ID AND PAPERS CITATIONS CSV PREVIOUS TO RELATIONS APPEND

df_cit = pd.DataFrame(papers_citations, columns=['PaperID', 'CitationID'])

df_cit.to_csv('./data/papers_citations.csv', index=False)

In [ ]:
#CITES
query = '''
LOAD CSV WITH HEADERS FROM "file:/C:/Users/odyky/Desktop/Property_Graph_Research_Publications/data/papers_citations.csv" AS x
MATCH (a:Paper {id: x.PaperID}), (b:Paper {id: x.CitationID})
MERGE (a)-[:cites]->(b)
'''

run_query(query)